Importing Libraries 

In [1]:
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf

Importing Data

In [2]:
x_input=np.load('D:\KAGGLE\hand sign number\Sign-language-digits-dataset\\X.npy')

In [3]:
y_output=np.load('D:\KAGGLE\hand sign number\Sign-language-digits-dataset\\Y.npy')

In [4]:
x_input=x_input.reshape(2062,64,64,1)

One-Hot Encoding

In [5]:
input_data=np.zeros((2062,64,64,1))
output_data=np.zeros((2062,10))
a=np.arange(2062)
a=np.random.permutation(a)
for i in range(0,2062):
    input_data[i,:,:,:]=x_input[a[i],:,:,:]
    output_data[i,:]=y_output[a[i],:]

In [6]:
x_train=input_data[0:1500,:,:,:]
y_train=output_data[0:1500,:]

In [7]:
m=x_train.shape[0]
n_h=x_train.shape[1]
n_w=x_train.shape[2]
n_c=x_train.shape[3]
tf.reset_default_graph()

In [8]:
x_train.shape

(1500, 64, 64, 1)

In [9]:
x=tf.placeholder(tf.float32,shape=[None,n_h,n_w,n_c])
y=tf.placeholder(tf.float32,shape=[None,10])

In [10]:
w1=tf.get_variable("w1",[4,4,1,8],initializer=tf.contrib.layers.xavier_initializer())
w2=tf.get_variable("w2",[2,2,8,16],initializer=tf.contrib.layers.xavier_initializer())

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Defining Model

In [11]:
z1=tf.nn.conv2d(x,w1,strides=[1,1,1,1],padding='SAME')
A1=tf.nn.relu(z1)
P1=tf.nn.max_pool(A1,ksize=[1,5,5,1],strides=[1,3,3,1],padding='SAME')
z2=tf.nn.conv2d(P1,w2,strides=[1,1,1,1],padding='SAME')
A2=tf.nn.relu(z2)
P2=tf.nn.max_pool(A2,ksize=[1,4,4,1],strides=[1,4,4,1],padding='SAME')
P2=tf.contrib.layers.flatten(P2)
z3=tf.contrib.layers.fully_connected(P2,num_outputs=10,activation_fn=None)

Instructions for updating:
Use keras.layers.flatten instead.


In [12]:
#label=tf.cast(output,tf.float32)
#logit=z3
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=z3,labels=y_train))

In [13]:
optimizer=tf.train.AdamOptimizer(0.009).minimize(cost)

# Training

In [14]:
init=tf.global_variables_initializer()

sess=tf.Session()
sess.run(init)
for i in range(1,300):
    opt=sess.run(optimizer,feed_dict={x:x_train,y:y_train})
    error=sess.run(cost,feed_dict={x:x_train,y:y_train})
    print(i,end=" ")
    print(error,end=" ")

1 2.3092043 2 2.3011265 3 2.2939067 4 2.285978 5 2.271613 6 2.2602613 7 2.237074 8 2.2095299 9 2.1720963 10 2.1221132 11 2.0562363 12 1.9852421 13 1.8983953 14 1.7971058 15 1.6851896 16 1.5651948 17 1.4430025 18 1.32798 19 1.2212863 20 1.1254283 21 1.0487721 22 0.9797542 23 0.92556244 24 0.8736443 25 0.82626325 26 0.7873201 27 0.7589526 28 0.7345817 29 0.69800514 30 0.6845565 31 0.6587821 32 0.6352923 33 0.6204432 34 0.61533827 35 0.5874058 36 0.5730271 37 0.55814505 38 0.54737145 39 0.5356519 40 0.5257629 41 0.51389366 42 0.5001511 43 0.48862305 44 0.47859737 45 0.46875012 46 0.4573244 47 0.4505904 48 0.44022688 49 0.42922246 50 0.4238047 51 0.41368 52 0.4071515 53 0.3983385 54 0.39235088 55 0.3830923 56 0.3775751 57 0.37076595 58 0.3643977 59 0.35785487 60 0.3522586 61 0.34568766 62 0.3395624 63 0.33419237 64 0.32881865 65 0.32400605 66 0.31989154 67 0.3213665 68 0.33960706 69 0.3532913 70 0.33960482 71 0.30080998 72 0.3212528 73 0.31834757 74 0.2934066 75 0.31088927 76 0.29241818 77

KeyboardInterrupt: 

# Training Accuracy

In [15]:
abc=sess.run(z3,feed_dict={x:x_train})


In [16]:
ab=np.array(abc.argmax(axis=1)).reshape(1,m)
m1=0
cd=np.array(y_train.argmax(axis=1)).reshape(1,m)

In [17]:
for i in range(0,m):
    if (ab[0,i]==cd[0,i]):
        m1=m1+1

In [18]:
m1

1474

training accuracy = 1474/1500

# Test Accuracy

In [19]:
abc=sess.run(z3,feed_dict={x:x_input})

In [20]:
ab=np.array(abc.argmax(axis=1)).reshape(1,2062)
m1=0
cd=np.array(y_output.argmax(axis=1)).reshape(1,2062)

In [21]:
for i in range(0,2062):
    if (ab[0,i]==cd[0,i]):
        m1=m1+1

In [22]:
m1

1965

test data accuracy= 1965/2062